In [2]:
import numpy as np
import pandas as pd

import csv

In [6]:
train = pd.read_csv('gapping/train.csv', sep = '\t', quoting = csv.QUOTE_NONE)
valid = pd.read_csv('gapping/dev.csv', sep = '\t', quoting = csv.QUOTE_NONE)
test = pd.read_csv('gapping/test.csv', sep = '\t', quoting = csv.QUOTE_NONE)

In [19]:
ptrain = train[train['class'] == 1]

In [20]:
pd.set_option('display.max_colwidth', 2000)
ptrain = ptrain.fillna('')

In [35]:
ptrain['tcR1'] = ptrain.apply(lambda row: row['text'][int(row['cR1'].split(':')[0]):int(row['cR1'].split(':')[1])] if row['cR1'] != '' else '', axis = 1)
ptrain['tcR2'] = ptrain.apply(lambda row: row['text'][int(row['cR2'].split(':')[0]):int(row['cR2'].split(':')[1])] if row['cR2'] != '' else '', axis = 1)

ptrain['tR1'] = ptrain.apply(lambda row: row['text'][int(row['R1'].split(':')[0]):int(row['R1'].split(':')[1])] if len(row['R1'].split(' ')) == 1 and row['R1'] != '' else '', axis = 1)
ptrain['tR2'] = ptrain.apply(lambda row: row['text'][int(row['R2'].split(':')[0]):int(row['R2'].split(':')[1])] if len(row['R2'].split(' ')) == 1 and row['R2'] != '' else '', axis = 1)

In [37]:
ptrain[['text', 'tcR1', 'tcR2', 'tR1', 'tR2']]

,text,tcR1,tcR2,tR1,tR2
1,"Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.",часть их,одного,часть,из двух
5,"Выворачивает наизнанку, запад превращает в юг, а север — в восток, меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.",запад,в юг,север,в восток
7,"Текущие обязательства компании в 2009 г. сократились на 16% - до 135.3 млн. грн., долгосрочные обязательства уменьшились на 1.7% - до 71.8 млн. грн., а дебиторская задолженность - на 3% до 87.7 млн. грн.",долгосрочные обязательства,на 1.7% - до 71.8 млн. грн.,дебиторская задолженность,на 3% до 87.7 млн. грн
8,"Первая конвенция была ратифицирована 80 процентами государств – членов МОТ, а вторая – 87 процентами этих государств.",Первая конвенция,80 процентами государств – членов МОТ,вторая,87 процентами этих государств
14,"Но Христианин сказал Упрямому: — Нет, сосед, лучше ты последуй примеру Сговорчивого. Мы в самом деле там получим то, о чем я сейчас говорил, а сверх того — великую славу.","то, о чем я сейчас говорил",,великую славу,
15,"Средний срок действия контрактов в 2000 году для консультантов увеличился на 16 дней, а для индивидуальных подрядчиков — на 15 дней.",для консультантов,на 16 дней,для индивидуальных подрядчиков,на 15 дней
20,"Однако за счет переориентации на другие рынки в целом импорт куриного мяса превысил показатели 2001 г. по стоимости - на 10,5%, по физическим объемам - на 1,4%.",по стоимости,"на 10,5%",по физическим объемам,"на 1,4%"
22,"В наше время скромность принимают за высокомерие, а доброту за слабость .",скромность,за высокомерие,доброту,за слабость
30,"В 2005 г. в газопроводы ЕСГ поступило 699,7 млрд м3 газа (в 2001 г. - 621,2 млрд м3), таким образом, объемы транспортировки газа выросли на 12,6 %.",В 2005 г.,"699,7 млрд м3 газа",в 2001 г.,"621,2 млрд м3"
34,"Закон состоит в том, что все вещества, которые разлагаются батареей, являются в жидком состоянии проводниками, а в твердом состоянии — непроводниками электричества гальванической батареи (393, 394, 404, 407, 413, 505, 676, 679, 697 и др).",в жидком состоянии,проводниками,в твердом состоянии,"непроводниками электричества гальванической батареи (393, 394, 404, 407, 413, 505, 676, 679, 697 и др)"


In [300]:
def find_left_part(sent, left_index):
    # принимает предложение и левый индекс; возвращает "левый предикат"
    rres = left_index
    bflag = 0
    for symbol in sent[:left_index][-1::-1]:
        if symbol.isalpha() or symbol.isdigit():
            bflag = 1
            break
        else:
            rres = rres - 1
    if bflag == 0:
        return '-1:-1'
    lres = rres
    last_digit = 1 if sent[rres].isdigit() else 0 # flag
    for symbol in sent[:rres][-1::-1]:
        if symbol in ['[', ']', '(', ')', ':', '^', "'", '"']:
            break
        elif (symbol in [',', '.']) and (last_digit == 0):
            break
        else:
            lres = lres - 1
            if symbol.isdigit():
                last_digit = 1
            else:
                last_digit = 0
    for symbol in sent[lres:rres]:
        if symbol.isalpha() or symbol.isdigit():
            break
        else:
            lres = lres + 1
    # далее удаление стартового союза, если такой имеется
    text = sent[lres:rres]
    cres = lres
    word = ''
    flag = 0
    for char in sent[lres:rres]:
        if char.isalpha():
            word = word + char
            cres = cres + 1
        else:
            flag = 1
            break
    if (flag == 1):
        morph = MorphAnalyzer()
        if morph.parse(word)[0].tag.POS == 'CONJ':
            lres = cres
            for symbol in sent[lres:rres]:
                if symbol.isalpha() or symbol.isdigit():
                    break
                else:
                    lres = lres + 1
    return f"{lres}:{rres}"

In [88]:
s = "Женщины становятся похожими на мужчин средней решимости, мужчины средней решимости — на женщин."
find_left_part(s, 84)

'57:82'

In [301]:
def find_right_part(sent, right_index):
    lres = right_index + 1
    bflag = 0
    for symbol in sent[lres:]:
        if symbol.isalpha() or symbol.isdigit():
            bflag = 1
            break
        else:
            lres = lres + 1
    if bflag == 0:
        return '-1:-1'
    rres = lres
    if lres == 0:
        last_dsep = 0
    else:
        last_dsep = 1 if (sent[lres-1] == ',' or sent[lres-1] == '.') else 0 # flag
    for symbol in sent[lres:]:
        if last_dsep == 1:
            if not symbol.isdigit():
                break
            else:
                rres = rres + 1
        if symbol in ['[', ']', '(', ')', ':', '^', "'", '"']:
            break
        elif symbol in [',', '.']:
            last_dsep = 1
        else:
            rres = rres + 1
            last_dsep = 0
    for symbol in sent[lres:rres][-1::-1]:
        if symbol.isalpha() or symbol.isdigit():
            break
        else:
            rres = rres - 1
    return f"{lres}:{rres}"

In [110]:
tmp = ptrain.iloc[0]

text     Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.
class                                                                                                                                          1
cV                                                                                                                                         92:99
cR1                                                                                                                                        83:91
cR2                                                                                                                                      103:109
V                                                                                                                                        127:127
R1                                                                                                                                

In [143]:
find_left_part(tmp['text'], int(tmp['V'].split(':')[0]))

'117:124'

In [144]:
tmp['text'][117:124]

'а часть'

In [178]:
train.iloc[16344]

text     Десятки женятся на десятках, восьмерки - на восьмерках, пятерки на пятерках и так далее.
class                                                                                           1
cV                                                                                           8:15
cR1                                                                                           0:7
cR2                                                                                         16:27
V                                                                                     41:41 64:64
R1                                                                                    29:38 56:63
R2                                                                                    41:54 64:75
Name: 16344, dtype: object

In [302]:
def find_r(row):
    if row['class'] == 0:
        row['cR1'] = '-1:-1'
        row['cR2'] = '-1:-1'
        row['R1'] = '-1:-1'
        row['R2'] = '-1:-1'
    else:
        res_r1 = ''
        res_r2 = ''
        for v_range in row['V'].strip().split(' '):
            res_r1 = res_r1 + find_left_part(row['text'], int(v_range.split(':')[0])) + ' '
            res_r2 = res_r2 + find_right_part(row['text'], int(v_range.split(':')[1]) - 1) + ' '
        row['cR1'] = find_left_part(row['text'], int(row['cV'].split(':')[0]))
        row['cR2'] = find_right_part(row['text'], int(row['cV'].split(':')[1]))
        row['R1'] = res_r1
        row['R2'] = res_r2
    return row

In [228]:
find_r(train.iloc[16344])

/home/vladimir/anaconda3/envs/dialogue2019gapping/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/vladimir/anaconda3/envs/dialogue2019gapping/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/vladimir/anaconda3/envs/dialogue2019gapping/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import k

text     Десятки женятся на десятках, восьмерки - на восьмерках, пятерки на пятерках и так далее.
class                                                                                           1
cV                                                                                           8:15
cR1                                                                                           0:7
cR2                                                                                         16:27
V                                                                                     41:41 64:64
R1                                                                                   29:38 56:63 
R2                                                                                   41:54 64:87 
Name: 16344, dtype: object

In [229]:
train.iloc[16344]['text'][29:38]

'восьмерки'

In [230]:
res = pd.read_csv('results/result.csv', sep = '\t', quoting = csv.QUOTE_NONE)

In [231]:
res.sample(10)

,text,class,cV,V
20893,"Сущность изобретения: способ получения водорастворимых азокрасителей для шерсти, кожи, полиамида, натурального шелка, дерева.",0,-1:-1,-1:-1
3609,"И чем сильнее я пытаюсь понять, что именно заставляет меня наслаждаться механикой, а их - ее ненавидеть, тем более неуловимым оно становится.",0,-1:-1,-1:-1
13905,"К тому же оно является «классическим», а 51% потребителей предпочитают печенье без глазури, 34% – без каких-либо вкусовых добавок или ароматизаторов.",0,-1:-1,-1:-1
16942,"Оставьте на полках несколько самых ценных для вас, а остальное спрячьте до лучших времен или раздарите друзьям.",0,-1:-1,-1:-1
8170,"Сущность изобретения: в антенне, содержащей включенные вдоль вертикального проводника токоотсекающие резонансные двухполюсники и трубчатое основание с согласующим блоком, в каждый токоотсекающий двухполюсник введен настроечный элемент, а согласующий элемент выполнен в виде цепочки из катушки индуктивности, конденсатора и отрезков длинных линий.",0,-1:-1,-1:-1
5746,"Если же хотя бына одном уровне контроль отсутствует, это уже не йога, а уходящая карма, которая машет на последок своей ручкой: ну посмотри ещё немного на свои желания, ну ещё немного помечтай...",0,-1:-1,-1:-1
19652,"Я была брезгливая, правда, мать еще надо мной сильно смеялась из-за этого, мол, никто больше не парится и вообще, что естественно, то не безобразно.",1,2:6,46:46 62:62
7539,"Поскольку команда как таковая еще не была сформирована, я смог начать свою работу достаточно эффективно, с помощью руководства и акционеров создавая новую команду с нуля.",0,-1:-1,-1:-1
9191,"Пускай лучшие лекари несут мой табут и все пусть видят, что даже самые лучшие лекари бессильны перед лицом смерти.",1,27:30,107:107
12992,"Сущность изобретения: система пожаротушения содержит блок снабжения энергией и подачи тушащего средства, блок контроля и управления, блок обнаружения и тушения пожара, магистральный трубопровод, распределительный трубопровод и включает несколько оросительных устройств с оросителями с замками и чувствительными элементами.",0,-1:-1,-1:-1


In [232]:
%%time

res.apply(find_r, axis = 1)

,text,class,cV,V,cR1,cR2,R1,R2
0,"В РП и торцевой стенке, расположенной со стороны ЭД, выполнены перепускные отверстия для прохода рабочей среды с возможностью сообщения с выпускным патрубком.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
1,"Диаграммы каротажа сопротивлений помогают в выделении вероятных переходных зон и контактов, а диаграммы плотностного и нейтронного каротажа — в определении пористости.",1,33:41,131:131 142:142,0:32,42:90,92:130 92:139,131:166 142:166
2,"При оценке структуры в тенговом эквиваленте 99,79% (99,81%) всего объема биржевых сделок приходится на USD; 0,09% (0,016%) -на EUR и 0,12% (0,17%) - на RUR.",1,89:99,144:144 149:149 149:149,60:88,100:112,140:144 140:144 140:144,149:155 149:155 149:155
3,Ты самый лучший во всём Мироздании и я люблю тебя всем сердцем и душой.,0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
4,"Способ позволяет ликвидировать обострение заболевания без применения антибиотиков и значительно сократить срок лечения, а также удлинить стадию ремиссии.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
5,"Изобретение относится к медицине, а именно к анатомии, применяется в изготовлении препаратов позвоночника.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
6,"— И если ты когда-нибудь почувствуешь себя одиноким, просто посмотри на луну, кто-то, где-то тоже смотрит на нее.",1,60:68,106:106,53:59,69:76,86:105,106:112
7,"Требуется:[соблюденте чистоты в комнате,своевременная оплата,в комнате не курить.]Все вопросы в Лс.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
8,"Обращает на себя внимание также тот факт, что среди образующихся экзо- и эндо-1-метилпенталанов преобладает экзо-изомер, тогда как экзо- и эндо-2-метилпенталаны образуются в сравнимых количествах.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
9,Слишком много твари и ради не стану и вапше перестала сами за ся и м мнени для сч,0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1


In [303]:
debug_res = pd.read_csv('debug/res.csv', sep ='\t', quoting = csv.QUOTE_NONE)
debug_res['text'] = valid['text']

In [304]:
debug_res = debug_res.apply(find_r, axis = 1)

In [305]:
debug_res[['text', 'class', 'cV', 'cR1', 'cR2', 'V', 'R1', 'R2']].to_csv('debug/debug_debug_debug_res.csv', sep = '\t', index = None, quoting=csv.QUOTE_NONE)

In [317]:
res = pd.read_csv('results/final_result.csv', sep = '\t', quoting = csv.QUOTE_NONE)

In [348]:
res.sample(20)

,text,class,cV,cR1,cR2,V,R1,R2
222,Как в сердце кого то пустить - если никто до него не дотронется..,0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
6568,"В 2002 г. по сравнению с 2001 г. общий экспорт Приднестровья сократился на 35.6%, в т.ч. в Молдову - на 39%.",1,61:71,33:60,72:79,101:101,89:98,101:106
15022,"Открытый плавательный бассейн , Открытый бассейн (работает круглый год.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
4140,"когда мы смотрим не на видимое, но на невидимое: ибо видимое временно, а невидимое вечно.",1,61:69,53:60,71:88,83:83,73:82,83:88
6202,"Такой выбор t и q обеспечивает выпаривание связки без разрушения алмазных (абразивных) зерен, которые при воздействии коротких импульсов нагреваются до температуры, не превышающей температуру графитизации алмаза.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
14332,"Несмотря на все это, NetSky.q продолжает оставаться одним из самых распространенных червей за всю историю интернета.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
3931,"У некоторых больных липодистрофия проявляется избыточным накоплением жира в определенных местах (обычно на животе и тыльной стороне шеи), а у других — напротив, уменьшением жировой ткани (на щеках, височной области, подкожной клетчатке конечностей).",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
9808,"Джерадин подвел ее к зеркалу и принялся водить свободной рукой, пытаясь установить его — или себя — в правильное положение.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
20030,"Его общие запасы увеличились на 7,6% (в основном за счет роста подтвержденных запасов - 7,5%), объем прогнозных ресурсов - на 2,7%, прирост забалансовых запасов составил 1%.",1,17:28,0:16,29:35,123:123,95:120,123:129
12320,"Однако, общая средневзвешенная доходность нот НБКР за месяц снизилась на 0,2 п.п., до 0,6 процента, в том числе: 7-дневных нот - с 0,4 до 0,3 процента, 14-дневнх нот - с 0,6 до 0,5 процента, а по 28-дневным нотам доходность снизилась на 0,2 п.п. до 0,8 процента.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1


In [353]:
res.iloc[6568]['text'][101:106]

'на 39'

In [280]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [284]:
'и пример'

[Parse(word='что', tag=OpencorporaTag('CONJ'), normal_form='что', score=0.791044, methods_stack=((<DictionaryAnalyzer>, 'что', 3046, 0),)),
 Parse(word='что', tag=OpencorporaTag('NPRO,neut sing,accs'), normal_form='что', score=0.089552, methods_stack=((<DictionaryAnalyzer>, 'что', 3047, 3),)),
 Parse(word='что', tag=OpencorporaTag('NPRO,neut sing,nomn'), normal_form='что', score=0.074626, methods_stack=((<DictionaryAnalyzer>, 'что', 3047, 0),)),
 Parse(word='что', tag=OpencorporaTag('PRCL'), normal_form='что', score=0.02985, methods_stack=((<DictionaryAnalyzer>, 'что', 22, 0),)),
 Parse(word='что', tag=OpencorporaTag('ADVB,Ques'), normal_form='что', score=0.014925, methods_stack=((<DictionaryAnalyzer>, 'что', 1130, 0),))]

In [299]:
morph.parse('а')[0].tag.POS

'CONJ'

In [308]:
debug_res

,text,class,cV,V,cR1,cR2,R1,R2
0,"Центральная часть лунной тени, где наблюдается полная фаза затмения вступила на Землю в Атлантике, пересекла южную часть африканского континента и крупный остров Мадагаскар, а завершила свой путь на закате в Индийском океане",1,25:29,206:206,0:24,31:97,176:205,206:224
1,"Я превращу твое сердце в траву, а все, что ты любишь, в овец.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
2,"В данном примере строки сгруппированы по названию товара, а столбцы по категории товаров.",1,24:37,68:68,0:23,38:56,60:67,68:88
3,"Ассоциация намерена занять на мировом рынке долю, равную долям ведущих мировых производителей",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
4,"""Ты сама все портишь""-твои слова хороший мой",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
5,"По его мнению, даже инспекции МАГАТЭ не всегда вовремя могут установить факт хищения или укрывательства ядерных материалов",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
6,"Сущность изобретения: в ранорасширитель введена съемная цилиндрическая дренирующая эластичная оболочка, а сам ранорасширитель выполнен в виде тороидального зигзагообразного элемента из материала с эффектом памяти формы",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
7,"Сущность изобретения: предлагается рыборазделочная машина, включающая первое разделочное устройство, в котором рыбы транспортируются поперек своей продольной оси, и второе разделочное устройство, которое присоединяется к первому по углом и в котором рыбы транспортируются в продольном направлении",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
8,"Заявления и комментарии, сделанные в ходе рассмотрения Комитетом этого пункта, отражены в соответствующем кратком отчете (А/С.5/45/SR.42",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1
9,"Жизнь их была открытой и эпикурейской, обряды посвящения — таинством, и верхи общин стремились вступить в это братство.",0,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1,-1:-1


In [316]:
debug_res.iloc[23]['text'][93:105]

'10 - сильную'